# Trabalho Final - Pesquisa Operacional <br>
## Aplicando Branch-and-Bound


Alunos:<br>
    João Victor Soares Silva, mat.: 20210027300 <br>
    Yhasmim de Souza Tigre, mat.: 20210026966 


Implementação 

In [ ]:
from mip import *
import numpy as np
import pandas as pd
import sys
import math

 A funcao objetivo sera de maximizacao e todas as restricoes serao de “menor ou igual”

O codigo de voces deve ramificar em torno da variavel xj cujo valor seja
fracionario e mais proximo de 0,5. Em um filho, adiciona-se a restricao xj = 1; no outro, a restricao xj = 0

Vamos usar Busca em Largura

* Executar
* Testar se as variáveis são binárias
* Caso não sejam: percorrer as variáveis e montar uma lista com as variáveis não binárias
* Pegar a variável mais próxima de 0,5 e formar mais dois modelos  filhos

* Quando encontrar uma solução inteira, guardar para comparar nos próximos modelos filhos (limite dual)